In [ ]:
import glob
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, multilabel_confusion_matrix
pd.set_option('display.max_columns', None)

events = pd.read_parquet("../data/gdelt/events/6_final/events_dataset_v2.parquet")
events = events[events['period'] <= '202406']

In [ ]:
print(len(events.index))
print(events.columns)
events.tail(3)

In [ ]:
# Forecasting Configuration
# ============================
# Set the forecast horizon: predict CS_score N periods ahead
# Options: 1 (nowcasting), 2, 3, or 4 (forecasting)
# Literature shows GDELT features are more valuable for forecasting (2-4 periods ahead)
# because baseline (previous_CS) becomes less powerful with longer horizons

FORECAST_HORIZON = 3  # Predict 3 periods ahead (can be changed to 2 or 4)

print(f"=== Forecasting Configuration ===")
print(f"Forecast Horizon: {FORECAST_HORIZON} period(s) ahead")
if FORECAST_HORIZON == 1:
    print("Mode: NOWCASTING (predicting next period)")
else:
    print(f"Mode: FORECASTING (predicting {FORECAST_HORIZON} periods ahead)")
print(f"\nWhy forecasting helps GDELT features:")
print(f"  - Baseline 'previous_CS' becomes less predictive ({FORECAST_HORIZON} periods old)")
print(f"  - GDELT temporal patterns can capture early warning signals")
print(f"  - More realistic for early warning systems\n")


In [ ]:
df = events.copy()
print(f"Total rows: {len(df.index)}")
print(f"Period range: {min(df['period'])} to {max(df['period'])}")
print(f"Columns: {len(df.columns)}")

# Drop rows where ADMIN0 is missing
mask_missing = df['ADMIN0'].isna() | (df['ADMIN0'] == 'None') | (df['ADMIN0'] == None)
if mask_missing.sum() > 0:
    print(f"Dropping {mask_missing.sum()} rows with ADMIN0=None")
    df = df[~mask_missing].copy()

print(f"Final rows: {len(df)}")
df.tail()

In [ ]:
key_cols = ["ADMIN0", "ADMIN1", "ADMIN2", "period"]

def check_duplicates(df, name):
    dupes = (
        df
        .groupby(key_cols)
        .size()
        .reset_index(name="n")
        .query("n > 1")
    )
    print(f"{name}: {len(dupes)} duplicated keys")
    return dupes

dupes = check_duplicates(df, "events")

In [ ]:
# BETWEEN-ASSESSMENT FORECASTING SETUP
# ====================================
# CRITICAL: This implements the correct "between-assessment forecasting" setup
# 
# Key principle: Observed IPC defines targets; unobserved months are prediction times, not missing labels.
#
# Target: y_next = next observed IPC assessment (forward-lagged label)
# Feature: IPC_last = last observed IPC before this row (forward-filled, constant between assessments)
#
# Rules:
# - Only use IPC_last as IPC feature (no rolling, no multiple lags, no trends)
# - CS_score is NOT the direct target (y_next is)
# - Keep ALL rows (including last periods without future assessments)
# - Filter to rows with valid y_next only when training models

print("="*70)
print("BETWEEN-ASSESSMENT FORECASTING SETUP")
print("="*70)
print("\nKey principle: Observed IPC defines targets; unobserved months are prediction times.\n")

# Create region identifiers
df['region'] = df['ADMIN0'] + '-' + df['ADMIN1']
df['district'] = df['ADMIN0'] + '-' + df['ADMIN1'] + '-' + df['ADMIN2']

# Sort by district (ADMIN2) and period to ensure deterministic operations
df = df.sort_values(['ADMIN0', 'ADMIN1', 'ADMIN2', 'period']).reset_index(drop=True)

# CS_score is already in the events dataset - just ensure numeric
df['CS_score'] = pd.to_numeric(df['CS_score'], errors='coerce')

# Create IPC_last: last observed IPC before this row (forward-filled)
# This is the ONLY IPC feature allowed - it's stale and constant between assessments
print("1. Creating IPC_last (last observed IPC, forward-filled)...")
df['IPC_last'] = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False)['CS_score'].transform(
    lambda x: x.ffill()
)

# Create y_next: next observed IPC assessment (forward-lagged label)
# This is the target variable - we predict the next assessment outcome
print("2. Creating y_next (next observed IPC assessment - THE TARGET)...")

def get_next_assessment(group):
    """Get the next observed IPC assessment for each row"""
    cs_values = group['CS_score'].values
    result = []
    
    for i in range(len(cs_values)):
        # Look forward to find next non-null value
        found = False
        for j in range(i + 1, len(cs_values)):
            if pd.notna(cs_values[j]) and 1 <= cs_values[j] <= 5:
                result.append(cs_values[j])
                found = True
                break
        if not found:
            result.append(np.nan)
    
    return pd.Series(result, index=group.index)

df['y_next'] = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False, group_keys=False).apply(
    get_next_assessment
)

# Keep ALL rows - don't drop periods without future assessments
# Rows without y_next (like 202402) can still be used for feature engineering
# Filter to rows with valid y_next only when training models
rows_with_y_next = df['y_next'].notna().sum()
rows_without_y_next = df['y_next'].isna().sum()
df = df[df['IPC_last'].isin([1, 2, 3, 4, 5])]
print(f"   Rows with y_next (can be used for training): {rows_with_y_next}")
print(f"   Rows without y_next (last periods, feature engineering only): {rows_without_y_next}")
print(f"   Total rows kept: {len(df)} (no periods dropped)")

# Clean y_next where it exists: ensure valid range (1-5) and convert to int
# Only clean where y_next is not NaN
mask_valid_y_next = df['y_next'].notna()
df.loc[mask_valid_y_next, 'y_next'] = df.loc[mask_valid_y_next, 'y_next'].round()
df.loc[mask_valid_y_next, 'y_next'] = df.loc[mask_valid_y_next, 'y_next'].clip(1, 5)
df.loc[mask_valid_y_next, 'y_next'] = df.loc[mask_valid_y_next, 'y_next'].astype(int)

# Summary statistics
print(f"\n=== Summary ===")
print(f"IPC_last: last observed IPC (forward-filled, constant between assessments)")
print(f"y_next: next observed IPC assessment (THE TARGET)")
print(f"\nIPC_last distribution:")
print(df['IPC_last'].value_counts().sort_index())
print(f"\ny_next distribution (TARGET):")
print(df['y_next'].value_counts().sort_index())
print(f"\nRows with valid y_next: {df['y_next'].notna().sum()}")
print(f"Rows without y_next (last periods): {df['y_next'].isna().sum()}")
print(f"Current DataFrame shape: {df.shape}")
print(f"Period range: {df['period'].min()} to {df['period'].max()}")

# Show example for verification
print(f"\n=== Example (first ADMIN2) ===")
if len(df) > 0:
    example_admin2 = df['ADMIN2'].iloc[0]
    example = df[df['ADMIN2'] == example_admin2].head(12)[['ADMIN2', 'period', 'CS_score', 'IPC_last', 'y_next']]
    print(example.to_string(index=False))
    print(f"\nNote: IPC_last is constant between assessments, y_next is the next assessment value")

print("\n" + "="*70)

In [ ]:
# Article coverage by IPC score and ADMIN0
# ==========================================

df_ipc = df[df['IPC_last'].notna()].copy()
df_ipc['has_articles'] = df_ipc['valid_SOURCEURL'].apply(
    lambda x: x.size > 0 if isinstance(x, np.ndarray) else bool(x) if x is not None else False
)

total = len(df_ipc)
print(f"Total rows (with IPC_last): {total:,}")
print(f"Rows with articles: {df_ipc['has_articles'].sum():,} ({df_ipc['has_articles'].mean()*100:.1f}%)")

# Breakdown by IPC_last
print("\nBreakdown by IPC_last:")
breakdown = df_ipc.groupby('IPC_last').agg(
    total=('has_articles', 'size'),
    with_articles=('has_articles', 'sum'),
)
breakdown['pct_articles'] = (breakdown['with_articles'] / breakdown['total'] * 100).round(1)
print(breakdown)

# Breakdown by ADMIN0
print("\nBreakdown by ADMIN0:")
breakdown_admin0 = df_ipc.groupby('ADMIN0').agg(
    total=('has_articles', 'size'),
    with_articles=('has_articles', 'sum'),
    admin2_total=('ADMIN2', 'nunique'),
)
admin2_cov = df_ipc[df_ipc['has_articles']].groupby('ADMIN0')['ADMIN2'].nunique().rename('admin2_with_coverage')
breakdown_admin0 = breakdown_admin0.join(admin2_cov, how='left').fillna({'admin2_with_coverage': 0})
breakdown_admin0['admin2_with_coverage'] = breakdown_admin0['admin2_with_coverage'].astype(int)
breakdown_admin0['pct_admin2_covered'] = (breakdown_admin0['admin2_with_coverage'] / breakdown_admin0['admin2_total'] * 100).round(1)
breakdown_admin0['pct_articles'] = (breakdown_admin0['with_articles'] / breakdown_admin0['total'] * 100).round(1)
breakdown_admin0 = breakdown_admin0.sort_values('total', ascending=False)
print(breakdown_admin0)

# Admin1 propagation gain
admin1_coverage = df_ipc.groupby(['ADMIN0', 'ADMIN1', 'period'])['has_articles'].transform('max').astype(bool)
o, p = df_ipc['has_articles'].sum(), admin1_coverage.sum()
print(f"\nAdmin1 propagation: {o:,} ({o/total*100:.1f}%) -> {p:,} ({p/total*100:.1f}%), gain +{p-o:,} ({(p-o)/total*100:.1f}%)")

# Breakdown by ADMIN0 after admin1 propagation
print("\nBreakdown by ADMIN0 (after admin1 propagation):")
df_ipc['has_articles_admin1'] = admin1_coverage.values
breakdown_prop = df_ipc.groupby('ADMIN0').agg(
    total=('has_articles', 'size'),
    admin2_total=('ADMIN2', 'nunique'),
    pct_admin2=('has_articles', 'mean'),
    pct_admin1=('has_articles_admin1', 'mean'),
)
breakdown_prop['pct_admin2'] = (breakdown_prop['pct_admin2'] * 100).round(1)
breakdown_prop['pct_admin1'] = (breakdown_prop['pct_admin1'] * 100).round(1)
breakdown_prop['gain_pp'] = (breakdown_prop['pct_admin1'] - breakdown_prop['pct_admin2']).round(1)
breakdown_prop = breakdown_prop.sort_values('total', ascending=False)
print(breakdown_prop)

In [ ]:
print(min(df['period']))
print(max(df['period']))
df = df.sort_values(['ADMIN0', 'ADMIN1', 'ADMIN2', 'period'])
df.tail()

### Feature engineering

In [ ]:
# COMPREHENSIVE FEATURE ENGINEERING
# ===================================
# This cell performs ALL feature engineering for IPC and GDELT Events features
# At this point: lists are NOT aggregated yet, y_next and IPC_last exist

print("="*70)
print("COMPREHENSIVE FEATURE ENGINEERING")
print("="*70)
print(f"\nStarting with {len(df)} rows")
print(f"Columns before feature engineering: {len(df.columns)}\n")

# ============================================================================
# 1. AGGREGATE GDELT LIST FEATURES
# ============================================================================
print("1. Aggregating GDELT list features...")

# Helper functions (already defined in previous cell, but ensure they exist)
def safe_list_agg(lst, func):
    """Safely aggregate a list, handling None, empty lists, and non-numeric values"""
    if lst is None:
        return np.nan
    if isinstance(lst, np.ndarray):
        if lst.size == 0:
            return np.nan
        lst = lst.tolist()
    try:
        if pd.isna(lst):
            return np.nan
    except (ValueError, TypeError):
        pass
    if isinstance(lst, (int, float)):
        return float(lst)
    if isinstance(lst, str):
        try:
            if lst.startswith('[') or lst.startswith('('):
                lst = eval(lst)
            else:
                return float(lst)
        except:
            return np.nan
    if not isinstance(lst, (list, tuple)):
        return np.nan
    if len(lst) == 0:
        return np.nan
    try:
        numeric_lst = []
        for x in lst:
            try:
                if pd.isna(x) or x is None:
                    continue
            except (ValueError, TypeError):
                pass
            try:
                val = float(x)
                if not np.isinf(val) and not np.isnan(val):
                    numeric_lst.append(val)
            except (ValueError, TypeError):
                continue
        if len(numeric_lst) == 0:
            return np.nan
        result = func(numeric_lst)
        return float(result) if not np.isnan(result) and not np.isinf(result) else np.nan
    except Exception as e:
        return np.nan

def safe_list_count(x):
    """Safely count elements in a list/array"""
    if x is None:
        return 0
    if isinstance(x, np.ndarray):
        return x.size if x.size > 0 else 0
    if isinstance(x, (list, tuple)):
        return len(x)
    try:
        if pd.isna(x):
            return 0
        return 1
    except (ValueError, TypeError):
        return 1

def aggregate_list_features(df, list_cols, prefix=""):
    """Aggregate list columns into meaningful statistical features for NLP-derived data
    
    For noisy NLP features, we keep only:
    - mean: central tendency
    - count: volume indicator
    - sum: total (especially useful for frequencies/counts)
    - max: peak intensity (only for crisis indicators)
    
    We skip: min (usually 0), std (variance not meaningful for noisy signals)
    """
    new_cols = {}
    cols_to_drop = []
    
    for col in list_cols:
        if col not in df.columns:
            continue
        
        base_name = col.replace('_list', '')
        if prefix:
            base_name = f"{prefix}_{base_name}"
        
        # For text columns (NER, clean_text), only create count
        if 'NER' in col or 'clean_text' in col:
            new_cols[f"{base_name}_count"] = df[col].apply(safe_list_count)
        else:
            # For numeric columns, create meaningful aggregations only
            # Mean: central tendency (most important for noisy signals)
            new_cols[f"{base_name}_mean"] = df[col].apply(lambda x: safe_list_agg(x, np.mean))
            
            # Count: volume indicator (how many articles/events)
            new_cols[f"{base_name}_count"] = df[col].apply(safe_list_count)
            
            # Sum: total (especially useful for frequencies and counts)
            new_cols[f"{base_name}_sum"] = df[col].apply(lambda x: safe_list_agg(x, np.sum))
            
            # Max: peak intensity (only for crisis indicators where peaks matter)
            # Check if this is a crisis-related feature
            is_crisis_feature = any(x in col.lower() for x in ['fatalities', 'displaced', 'injured', 
                                                                 'violence', 'torture', 'crisis'])
            if is_crisis_feature:
                new_cols[f"{base_name}_max"] = df[col].apply(lambda x: safe_list_agg(x, np.max))
            
            # Skip: min (usually 0), std (variance not meaningful for noisy NLP signals)
        
        cols_to_drop.append(col)
    
    if new_cols:
        new_df = pd.DataFrame(new_cols, index=df.index)
        df = pd.concat([df, new_df], axis=1)
    
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
    
    return df

# Find and aggregate list columns (events only, no merge suffix)
list_cols = [c for c in df.columns if '_list' in c]
print(f"   Found {len(list_cols)} list columns")

if list_cols:
    print("   Aggregating events features...")
    df = aggregate_list_features(df, list_cols, prefix="evt")

# ============================================================================
# 2. CLEAN AND PREPARE IPC FEATURES
# ============================================================================
print("\n2. Cleaning IPC features...")

# Clean IPC_last: round to nearest integer and ensure valid range
df['IPC_last'] = df['IPC_last'].round()
# Keep IPC_last as float for now (can filter invalid values later if needed)
# Don't drop rows here - keep all periods

print(f"   IPC_last range: {df['IPC_last'].min():.1f} to {df['IPC_last'].max():.1f}")
print(f"   Valid IPC_last (1-5): {(df['IPC_last'].between(1, 5, inclusive='both')).sum()} rows")

# ============================================================================
# 3. CREATE TEMPORAL FEATURES FOR GDELT
# ============================================================================
print("\n3. Creating temporal features for GDELT...")

# Focus on most reliable features for temporal patterns
# Prioritize: sentiment, crisis indicators, food security
key_gdelt_features = [
    # Sentiment (most reliable NLP signals)
    'evt_compound_score_mean', 'evt_sentiment.compound_mean',
    
    # Crisis indicators (high signal-to-noise)
    'evt_fatalities_freq_mean', 'evt_displaced_freq_mean',
    
    # Food security (directly relevant to IPC)
    'evt_food_insecurity_freq_mean',
    
    # Economic/agricultural (relevant but less reliable)
    'evt_economic_shocks_freq_mean', 'evt_agriculture_freq_mean',
]

# Filter to features that actually exist
key_gdelt_features = [f for f in key_gdelt_features if f in df.columns]

print(f"   Creating temporal features for {len(key_gdelt_features)} key GDELT features")

# Ensure sorted for lag operations
df = df.sort_values(['ADMIN0', 'ADMIN1', 'ADMIN2', 'period']).reset_index(drop=True)

# Create rolling windows (smoothing noise - most useful for NLP features)
# Use 3 and 6 periods to capture short and medium-term trends
for window in [3, 6]:
    for feat in key_gdelt_features:
        df[f"{feat}_rolling_{window}"] = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False)[feat].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )

# Create lags (1 and 2 periods only - reduce noise)
# Lags can capture trends but too many lags add noise
for lag in [1, 2]:
    for feat in key_gdelt_features:
        df[f"{feat}_lag{lag}"] = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False)[feat].shift(lag)

# Create escalation indicators (binary - less noisy than continuous change)
# Use rolling mean comparison instead of raw change to reduce noise
for feat in key_gdelt_features:
    rolling_3 = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False)[feat].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )
    rolling_6 = df.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'], sort=False)[feat].transform(
        lambda x: x.rolling(window=6, min_periods=2).mean()
    )
    # Escalation: current 3-period average > 6-period average
    df[f"{feat}_escalation"] = (rolling_3 > rolling_6).astype(int)

# Skip: change, pct_change, anomaly (too noisy for NLP-derived features)

print(f"   Created temporal features: rolling windows (3, 6 periods), lags (1, 2 periods), escalation indicators")

# ============================================================================
# 4. CREATE FEATURE INDICATORS AND INTERACTIONS
# ============================================================================
print("\n4. Creating feature indicators and interactions...")

# GDELT feature availability indicator
gdelt_cols = [c for c in df.columns if c.startswith('evt_')]
if gdelt_cols:
    has_gdelt = df[gdelt_cols].notna().any(axis=1)
    df['has_gdelt'] = has_gdelt.astype(int)
else:
    df['has_gdelt'] = 0

# Sentiment features from events

# Combine crisis indicators
crisis_features = [c for c in df.columns if any(x in c for x in ['fatalities', 'displaced', 'injured']) 
                   and c.endswith('_mean') and c.startswith('evt_')]
if crisis_features:
    df['crisis_intensity'] = df[crisis_features].fillna(0).sum(axis=1)

# Food security combined
food_features = [c for c in df.columns if 'food_insecurity' in c and c.endswith('_mean') 
                 and c.startswith('evt_')]
if food_features:
    df['food_security_combined'] = df[food_features].fillna(0).mean(axis=1)

print(f"   Created interaction and combined features")

# ============================================================================
# 5. CREATE GEOGRAPHIC DUMMY VARIABLES
# ============================================================================
print("\n5. Creating geographic dummy variables...")

# Only create dummies for base geographic identifiers (ADMIN0, ADMIN1, ADMIN2)
# Note: 'region' and 'district' are redundant (derived from ADMIN0+ADMIN1 and ADMIN0+ADMIN1+ADMIN2)
geographic_cols = []
for col in ['ADMIN0', 'ADMIN1', 'ADMIN2']:
    if col in df.columns:
        # Create dummy variables (one-hot encoding)
        dummies = pd.get_dummies(df[col], prefix=col, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        geographic_cols.extend(dummies.columns.tolist())
        print(f"   Created {len(dummies.columns)} dummy variables for {col}")

geographic_features = geographic_cols
print(f"   Total geographic features: {len(geographic_features)}")
print(f"   Note: 'region' and 'district' are kept as categorical identifiers but not converted to dummies (redundant with ADMIN0+ADMIN1+ADMIN2)")

# ============================================================================
# 6. SUMMARY
# ============================================================================
print("\n" + "="*70)
print("FEATURE ENGINEERING COMPLETE")
print("="*70)

print(f"\nFinal dataset: {len(df)} rows")
print(f"Total columns: {len(df.columns)}")

# Feature counts
ipc_features = ['IPC_last']

# GDELT base features: evt_ prefixed features
gdelt_base = [c for c in df.columns if c.startswith('evt_') 
              and not any(x in c for x in ['_lag', '_rolling', '_escalation'])]

# GDELT temporal features: evt_ prefixed features with temporal patterns
gdelt_temporal = [c for c in df.columns if c.startswith('evt_') 
                   and any(x in c for x in ['_lag', '_rolling', '_escalation'])]

# Geographic features were created as dummies in section 5 above
# geographic_features is already set to geographic_cols

# Geographic features: only ADMIN0, ADMIN1, ADMIN2 are converted to dummies
# 'region' and 'district' are kept as categorical identifiers for grouping but not converted to dummies (redundant)
# Exclude original categorical columns from feature lists (but keep them in dataframe for grouping purposes)
geographic_categorical_cols = ['ADMIN0', 'ADMIN1', 'ADMIN2', 'region', 'district']

# Exclude all non-feature columns (geographic categoricals, metadata, scores, indicators, target, list columns)
excluded_cols = (ipc_features + gdelt_base + gdelt_temporal + geographic_features +
                 ['y_next', 'has_gdelt', 'has_articles',
                  'CS_score',
                  'period', 'SQLDATE', 'EventCode', 'SOURCEURL', 'NumMentions', 'NumSources', 
                  'NumArticles', 'valid_SOURCEURL'] +
                 geographic_categorical_cols +
                 # Also exclude any remaining list columns that weren't aggregated
                 [c for c in df.columns if '_list' in c])
other_features = [c for c in df.columns if c not in excluded_cols]

print(f"\nFeature breakdown:")
print(f"  - IPC features: {len(ipc_features)}")
print(f"  - GDELT base features: {len(gdelt_base)} (mean, count, sum, max for crisis)")
print(f"  - GDELT temporal features: {len(gdelt_temporal)} (rolling windows, lags, escalation)")
print(f"  - Geographic features: {len(geographic_features)} (dummy variables)")
print(f"  - Other features: {len(other_features)}")
print(f"  - Total features: {len(ipc_features) + len(gdelt_base) + len(gdelt_temporal) + len(geographic_features) + len(other_features)}")
print(f"\nNote: Simplified feature set for noisy NLP-derived signals:")
print(f"  - Base: mean, count, sum (all features) + max (crisis features only)")
print(f"  - Temporal: rolling windows (3, 6 periods), lags (1, 2 periods), escalation indicators")
print(f"  - Removed: min, std, change, pct_change, anomaly (too noisy for NLP features)")

print(f"\nRows with y_next: {df['y_next'].notna().sum()} ({df['y_next'].notna().mean()*100:.1f}%)")
print(f"Rows with IPC_last: {df['IPC_last'].notna().sum()} ({df['IPC_last'].notna().mean()*100:.1f}%)")
print(f"Rows with GDELT features: {df['has_gdelt'].sum()} ({df['has_gdelt'].mean()*100:.1f}%)")

# ============================================================================
# 6. DROP PROCESSED/METADATA COLUMNS
# ============================================================================
print("\n6. Dropping processed metadata columns...")

cols_to_drop = ['SQLDATE', 'EventCode', 'SOURCEURL', 'NumMentions', 
                'NumSources', 'NumArticles', 'valid_SOURCEURL']
# Only drop columns that exist
cols_to_drop = [c for c in cols_to_drop if c in df.columns]
if cols_to_drop:
    df.drop(cols_to_drop, axis=1, inplace=True)
    print(f"   Dropped {len(cols_to_drop)} metadata columns")

print("\n" + "="*70)
print("READY FOR MODELING")
print("="*70)
print("\nFeature sets available:")
print(f"  - IPC only: {len(ipc_features)} IPC features + {len(geographic_features)} geographic features")
print(f"  - IPC + GDELT base: {len(ipc_features)} IPC + {len(gdelt_base)} GDELT base + {len(geographic_features)} geographic")
print(f"  - IPC + GDELT + temporal: {len(ipc_features)} IPC + {len(gdelt_base)} GDELT base + {len(gdelt_temporal)} GDELT temporal + {len(geographic_features)} geographic")
print("="*70)

In [ ]:
print(min(df['period']))
print(max(df['period']))
df = df.sort_values(['ADMIN0', 'ADMIN1', 'ADMIN2', 'period'])
df.tail()

### Configure train and test, and loops

In [ ]:
# ============================================================================
# NAIVE BASELINE MODELS
# ============================================================================
# PPS     — Previous Period Score: predict y_next = IPC_last (no-change)
# SPLY    — Same Period Last Year: CS_score from ~12 months ago
# Max-2PP — Max of 2 Previous Periods: max CS_score of last 2 assessments
#
# Evaluated on every assessment period that has y_next data, then averaged.
# ============================================================================

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

admin_cols = ['ADMIN0', 'ADMIN1', 'ADMIN2']

# All assessment periods with valid CS_score data
eval_periods = sorted(
    df.loc[df['CS_score'].between(1, 5), 'period'].unique()
)

print(f"Evaluating baselines on {len(eval_periods)} assessment periods: {eval_periods[0]} → {eval_periods[-1]}\n")

# ── Evaluate each baseline across all periods ────────────────────────────────
baseline_names = ['PPS', 'SPLY', 'Max-2PP']
period_results = {name: [] for name in baseline_names}

for test_period in eval_periods:
    test = df[(df['period'] == test_period) & df['CS_score'].between(1, 5)].copy()
    train = df[(df['period'] < test_period) & df['CS_score'].between(1, 5)].copy()
    train_periods = sorted(train['period'].unique())

    if len(test) == 0 or len(train_periods) < 2:
        continue

    y_test = test['CS_score'].astype(int)

    # PPS: previous assessment's CS_score (last training period, NOT current)
    pps_lookup = (train[train['period'] == train_periods[-1]][admin_cols + ['CS_score']]
                  .drop_duplicates(admin_cols))
    pps_merged = test.merge(pps_lookup.rename(columns={'CS_score': 'pred'}),
                            on=admin_cols, how='left')
    pps_pred = pps_merged['pred'].fillna(0).astype(int)

    # SPLY: CS_score from closest assessment ~12 months before test
    target_sply = str(int(test_period) - 100)
    sply_period = min(train_periods, key=lambda p: abs(int(p) - int(target_sply)))
    sply_lookup = (train[train['period'] == sply_period][admin_cols + ['CS_score']]
                   .drop_duplicates(admin_cols))
    sply_merged = test.merge(sply_lookup.rename(columns={'CS_score': 'pred'}),
                             on=admin_cols, how='left')
    sply_pred = sply_merged['pred'].fillna(0).astype(int)

    # Max-2PP: max CS_score from last 2 assessment periods
    last2 = train_periods[-2:]
    max2pp_lookup = (train[train['period'].isin(last2)]
                     .groupby(admin_cols)['CS_score'].max().reset_index())
    max2pp_merged = test.merge(max2pp_lookup.rename(columns={'CS_score': 'pred'}),
                               on=admin_cols, how='left')
    max2pp_pred = max2pp_merged['pred'].fillna(0).astype(int)

    preds = {'PPS': pps_pred, 'SPLY': sply_pred, 'Max-2PP': max2pp_pred}

    for name in baseline_names:
        period_results[name].append({
            'period': test_period,
            'n_test': len(test),
            'accuracy': accuracy_score(y_test, preds[name]),
            'precision': precision_score(y_test, preds[name], average='weighted', zero_division=0),
            'recall': recall_score(y_test, preds[name], average='weighted', zero_division=0),
            'f1': f1_score(y_test, preds[name], average='weighted', zero_division=0),
        })

# ── Summary ──────────────────────────────────────────────────────────────────
rows = []
for name in baseline_names:
    res = period_results[name]
    rows.append({
        'Model': name,
        'Accuracy': np.mean([r['accuracy'] for r in res]),
        'Precision': np.mean([r['precision'] for r in res]),
        'Recall': np.mean([r['recall'] for r in res]),
        'F1': np.mean([r['f1'] for r in res]),
        'N Periods': len(res),
        'Total Test': sum(r['n_test'] for r in res),
    })

baseline_summary = pd.DataFrame(rows)
print(baseline_summary.to_string(index=False))

# ML from here

In [ ]:
n_training_periods = 20     # months of training data before each test assessment
n_eval_periods = 3          # number of IPC assessment periods to evaluate (backwards from latest)
min_obs_per_assessment = 50 # minimum admin2 areas scored to qualify as an assessment period

# Optional: set to None to auto-detect latest assessment, or specify e.g. 202407
start_period = None

In [ ]:
# ============================================================================
# ROLLING WINDOW EVALUATION — TEST ON IPC ASSESSMENT PERIODS
# ============================================================================
# Strategy:
#   - Identify IPC assessment periods (months where FEWS NET published scores
#     for >= min_obs_per_assessment admin2 areas).
#   - Test on each assessment period: rows with valid y_next → predict next IPC.
#   - Train on all prior periods (up to n_training_periods months before).
#   - No leakage: IPC_last tells the model the *current* assessment;
#     y_next is the *future* assessment — different values.
# ============================================================================

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from catboost import CatBoostClassifier

all_periods = sorted(df['period'].unique())

# ── 1. Identify IPC assessment periods ──────────────────────────────────────
ipc_obs_per_period = df[df['CS_score'].between(1, 5)].groupby('period').size()
assessment_periods = sorted(ipc_obs_per_period.index.tolist())

print("="*70)
print("IPC ASSESSMENT PERIODS")
print("="*70)
print(f"\nTotal periods in data: {len(all_periods)}  ({all_periods[0]} → {all_periods[-1]})")
print(f"Assessment periods (with CS_score data): {len(assessment_periods)}\n")
for p in assessment_periods:
    print(f"  {p}:  {ipc_obs_per_period[p]:>5,} scored")

# ── 2. Select evaluation assessment periods (backwards from latest) ─────────
if start_period is not None:
    candidates = [p for p in assessment_periods if p <= start_period]
else:
    candidates = list(assessment_periods)

eval_assessments = candidates[-n_eval_periods:]  # latest N

print(f"\n{'='*70}")
print("ROLLING WINDOW EVALUATION SETUP")
print(f"{'='*70}")
print(f"\nConfiguration:")
print(f"  - Training window:       {n_training_periods} periods before test assessment")
print(f"  - Assessments to test:   {len(eval_assessments)} (of {len(assessment_periods)} total)")
print(f"  - Assessment periods:    {len(assessment_periods)}")

# ── 3. Create evaluation splits ─────────────────────────────────────────────
evaluation_splits = []

for i, test_period in enumerate(reversed(eval_assessments)):
    test_idx = all_periods.index(test_period)
    train_end_idx = test_idx - 1  # everything BEFORE test period
    train_start_idx = max(0, train_end_idx - n_training_periods + 1)

    if train_end_idx < 0:
        print(f"\n  ⚠️  Assessment {test_period}: no training data available — skipping")
        continue

    train_periods = all_periods[train_start_idx:train_end_idx + 1]

    evaluation_splits.append({
        'eval_id': i + 1,
        'test_period': test_period,
        'n_scored': int(ipc_obs_per_period.get(test_period, 0)),
        'train_periods': train_periods,
        'train_start': train_periods[0],
        'train_end': train_periods[-1],
        'n_train_periods': len(train_periods),
    })

print(f"\nEvaluation schedule (latest → earliest):\n")
for s in evaluation_splits:
    print(f"  Eval {s['eval_id']}: test {s['test_period']}  "
          f"({s['n_scored']:,} scored)  |  "
          f"train {s['train_start']} → {s['train_end']}  "
          f"({s['n_train_periods']} periods)")

print(f"\n{'='*70}")
print(f"Created {len(evaluation_splits)} evaluation splits")
print("="*70)

# ============================================================================
# DEFINE FEATURE SETS
# ============================================================================
if 'ipc_features' in globals():
    ipc_feat = [c for c in ipc_features if c in df.columns]
else:
    ipc_feat = ['IPC_last']

_temporal_suffixes = ('_rolling_', '_lag', '_escalation')

if 'gdelt_features' in globals():
    gdelt_base_feat = [c for c in gdelt_features if c in df.columns]
else:
    gdelt_base_feat = [c for c in df.columns
                       if c.startswith('evt_') and not any(s in c for s in _temporal_suffixes)]

if 'temporal_features' in globals():
    gdelt_temporal_feat = [c for c in temporal_features if c in df.columns]
else:
    gdelt_temporal_feat = [c for c in df.columns
                           if c.startswith('evt_') and any(s in c for s in _temporal_suffixes)]

if 'geographic_features' in globals():
    geographic_feat = [c for c in geographic_features if c in df.columns]
else:
    geographic_feat = [c for c in df.columns if c.startswith('geo_')]

feature_sets = {
    'IPC only':               ipc_feat + geographic_feat,
    'IPC + GDELT base':       ipc_feat + gdelt_base_feat + geographic_feat,
    'IPC + GDELT + temporal': ipc_feat + gdelt_base_feat + gdelt_temporal_feat + geographic_feat,
}

print("\nFeature sets available:")
for name, features in feature_sets.items():
    n_ipc = len([c for c in features if c in ipc_feat])
    n_gdelt_base = len([c for c in features if c in gdelt_base_feat])
    n_gdelt_temp = len([c for c in features if c in gdelt_temporal_feat])
    n_geo = len([c for c in features if c in geographic_feat])
    print(f"  - {name}: {n_ipc} IPC + {n_gdelt_base} GDELT base + {n_gdelt_temp} GDELT temporal + {n_geo} geographic = {len(features)} total")

# ============================================================================
# DEFINE MODELS
# ============================================================================
models = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs'),
        'needs_scaling': True,
    },
    'Random Forest': {
        'model': RandomForestClassifier(
            n_estimators=300, max_depth=None, min_samples_split=5,
            min_samples_leaf=2, class_weight='balanced', random_state=42, n_jobs=-1
        ),
        'needs_scaling': False,
    },
    'CatBoost': {
        'model': CatBoostClassifier(
            iterations=500, depth=6, learning_rate=0.1,
            loss_function='MultiClass', auto_class_weights='Balanced',
            random_seed=42, verbose=0
        ),
        'needs_scaling': False,
    },
}

print(f"\nModels to evaluate: {', '.join(models.keys())}")

# ============================================================================
# TRAIN AND EVALUATE
# ============================================================================
print("\n" + "="*70)
print("TRAINING AND EVALUATION")
print("="*70)

all_results = {}  # keyed by (model_name, feature_set_name)

for model_name, model_cfg in models.items():
    for feature_set_name, feature_cols in feature_sets.items():
        result_key = (model_name, feature_set_name)
        print(f"\n{'='*70}")
        print(f"MODEL: {model_name}  |  FEATURE SET: {feature_set_name}")
        print(f"{'='*70}")
        print(f"Using {len(feature_cols)} features")

        feature_results = []

        for split in evaluation_splits:
            print(f"\n  ── Eval {split['eval_id']}: Assessment period {split['test_period']} ──")

            # ── Training set: all prior periods with valid y_next ────────
            #    (includes inter-assessment rows with GDELT features)
            train_df = df[
                (df['period'].isin(split['train_periods'])) &
                (df['y_next'].notna())
            ].copy()

            # ── Test set: rows from assessment period with valid CS_score ─
            test_df = df[
                (df['period'] == split['test_period']) &
                (df['CS_score'].between(1, 5))
            ].copy()

            print(f"  Training samples:   {len(train_df):,}")
            print(f"  Test samples:       {len(test_df):,}  (admin2 areas in {split['test_period']})")

            if len(train_df) == 0:
                print(f"  ⚠️  Skipping — no training data")
                continue
            if len(test_df) == 0:
                print(f"  ⚠️  Skipping — no test data")
                continue

            # ── Prepare features and target ──────────────────────────────
            X_train = train_df[feature_cols].fillna(0)
            y_train = train_df['y_next'].astype(int)

            X_test = test_df[feature_cols].fillna(0)
            y_test = test_df['CS_score'].astype(int)

            # ── Fix IPC_last leakage: at assessment periods IPC_last == CS_score,
            #    so replace it with the previous assessment's score ────────
            if 'IPC_last' in feature_cols:
                ipc_col_idx = feature_cols.index('IPC_last')
                # Find previous assessment period in training window
                train_assessments = sorted(
                    p for p in assessment_periods if p in split['train_periods']
                )
                if train_assessments:
                    prev_period = train_assessments[-1]
                    prev_ipc = (
                        df.loc[(df['period'] == prev_period) & df['CS_score'].between(1, 5),
                               ['ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score']]
                        .drop_duplicates(['ADMIN0', 'ADMIN1', 'ADMIN2'])
                        .rename(columns={'CS_score': '_prev_ipc'})
                    )
                    test_with_prev = test_df.merge(prev_ipc, on=['ADMIN0', 'ADMIN1', 'ADMIN2'], how='left')
                    if isinstance(X_test, np.ndarray):
                        X_test[:, ipc_col_idx] = test_with_prev['_prev_ipc'].fillna(0).values
                    else:
                        X_test['IPC_last'] = test_with_prev['_prev_ipc'].fillna(0).values

            # ── Scale (only for models that need it) ─────────────────────
            if model_cfg['needs_scaling']:
                scaler = StandardScaler()
                X_train_fit = scaler.fit_transform(X_train)
                X_test_fit = scaler.transform(X_test)
            else:
                X_train_fit = X_train
                X_test_fit = X_test

            # ── Train ────────────────────────────────────────────────────
            print(f"  Training {model_name}...")
            model = clone(model_cfg['model'])
            model.fit(X_train_fit, y_train)

            y_pred = model.predict(X_test_fit)

            # ── Metrics ──────────────────────────────────────────────────
            accuracy  = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            recall    = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1        = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            cm        = confusion_matrix(y_test, y_pred)

            feature_results.append({
                'eval_id': split['eval_id'],
                'test_period': split['test_period'],
                'n_train': len(train_df),
                'n_test': len(test_df),
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'confusion_matrix': cm
            })

            print(f"\n  Results  (n_test={len(test_df):,}):")
            print(f"    Accuracy:  {accuracy:.4f}")
            print(f"    Precision: {precision:.4f}")
            print(f"    Recall:    {recall:.4f}")
            print(f"    F1 Score:  {f1:.4f}")
            print(f"\n  Confusion Matrix:\n    {cm}")

        all_results[result_key] = feature_results

# ============================================================================
# SUMMARY
# ============================================================================
if any(len(v) > 0 for v in all_results.values()):
    print("\n" + "="*70)
    print("SUMMARY: AVERAGE RESULTS BY MODEL & FEATURE SET")
    print("="*70)

    summary_data = []
    for (model_name, feature_set_name), results in all_results.items():
        if len(results) == 0:
            continue

        avg_acc  = np.mean([r['accuracy']  for r in results])
        avg_prec = np.mean([r['precision'] for r in results])
        avg_rec  = np.mean([r['recall']    for r in results])
        avg_f1   = np.mean([r['f1']        for r in results])
        total_test = sum(r['n_test'] for r in results)

        print(f"\n{model_name} — {feature_set_name}:")
        print("-" * 70)
        print(f"  Average Metrics (across {len(results)} assessments, {total_test:,} total test samples):")
        print(f"    Accuracy:  {avg_acc:.4f}")
        print(f"    Precision: {avg_prec:.4f}")
        print(f"    Recall:    {avg_rec:.4f}")
        print(f"    F1 Score:  {avg_f1:.4f}")

        # Per-evaluation breakdown
        print(f"\n  Per assessment:")
        for r in results:
            print(f"    {r['test_period']}:  n={r['n_test']:>5,}  acc={r['accuracy']:.3f}  f1={r['f1']:.3f}")

        n_features = len(feature_sets[feature_set_name])
        n_ipc = len([c for c in feature_sets[feature_set_name] if c in ipc_feat])
        n_gdelt_base = len([c for c in feature_sets[feature_set_name] if c in gdelt_base_feat])
        n_gdelt_temp = len([c for c in feature_sets[feature_set_name] if c in gdelt_temporal_feat])
        n_geo = len([c for c in feature_sets[feature_set_name] if c in geographic_feat])

        summary_data.append({
            'Model': model_name,
            'Feature Set': feature_set_name,
            'N Features': n_features,
            'IPC': n_ipc,
            'GDELT Base': n_gdelt_base,
            'GDELT Temporal': n_gdelt_temp,
            'Geographic': n_geo,
            'Accuracy': avg_acc,
            'Precision': avg_prec,
            'Recall': avg_rec,
            'F1 Score': avg_f1,
            'Total Test': total_test,
            'N Evals': len(results),
        })

    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('F1 Score', ascending=False)

    print(f"\n{'='*70}")
    print("SUMMARY TABLE")
    print("="*70)
    print(summary_df.to_string(index=False))

else:
    print("\n⚠️  No evaluations completed — check data availability")

# Why Literature Shows Better Results: Key Differences

## Potential Reasons for Better Performance in Literature:

1. **Different Prediction Tasks**:
   - **Transitions/Changes**: Predicting when CS_score will worsen (crisis onset) rather than exact level
   - **Early Warning**: Predicting 2-4 periods ahead (forecasting) rather than next period (nowcasting)
   - **Binary Classification**: Predicting crisis (CS≥3) vs non-crisis (CS<3) rather than 5-class classification

2. **Feature Engineering**:
   - **Temporal Patterns**: Using lagged and rolling features (as we just added) rather than only current-period values
   - **Anomaly Detection**: Focusing on deviations from historical patterns
   - **Interaction Features**: Combining GDELT with other data sources (weather, prices, etc.)

3. **Data Coverage**:
   - **Spatial Aggregation**: Some studies aggregate to ADMIN0 or ADMIN1 level where coverage is better
   - **Temporal Aggregation**: Using quarterly rather than monthly data
   - **Filtering**: Focusing on regions/periods with good GDELT coverage

4. **Evaluation Metrics**:
   - **Recall for Rare Events**: Focusing on detecting crises (high recall for CS≥3) rather than overall accuracy
   - **Early Detection**: Measuring how early they detect transitions, not just accuracy

5. **Baseline Comparison**:
   - **Weaker Baselines**: Some studies don't include `previous_CS` in baseline, making improvement easier to show
   - **Different Baselines**: Comparing against simpler models or using different evaluation windows

## Our Current Approach:
- ✅ Now includes temporal GDELT features (lags, rolling windows, anomalies, escalation)
- ✅ Uses same framework as previous work (allowing fair comparison)
- ⚠️ Still predicting exact CS_score level (highly autocorrelated)
- ⚠️ Using monthly data with CS_score available every 4 months

## Potential Improvements:
1. **Predict Transitions**: Predict if CS_score will worsen (CS_t+1 > CS_t) rather than exact value
2. **Forecasting Horizon**: Predict 2-4 periods ahead instead of next period
3. **Crisis Detection**: Binary classification (crisis vs non-crisis) with focus on recall
4. **Feature Selection**: Use only temporal GDELT features, filter out low-importance current-period features


## Feature Engineering Summary

### Features Created:

1. **List Aggregations** (from events):
   - Mean, Max, Sum, Count for all list features
   - Prefixed with `evt_`

2. **Derived Features**:
   - Crisis severity indicators
   - Coverage intensity metrics

3. **Temporal Features**:
   - Lag features (CS_score_lag1, lag2, lag3)
   - Moving averages (ma2, ma3)
   - Change and percentage change features
   - Cyclical period encoding (sin/cos)

4. **Geographic Features**:
   - Aggregated CS_score by ADMIN0 and ADMIN1 levels
   - Standard deviations by geographic level
   - Count of regions with same score

5. **Interaction Features**:
   - Ratios (casualty_rate, aid_per_casualty)
   - Normalized features (tone_abs_normalized)
   - Coverage metrics (mentions_per_source, articles_per_source)

### Next Steps for ML:

1. **Feature Selection**: Consider using:
   - Correlation-based selection
   - Mutual information
   - Recursive feature elimination
   - L1 regularization (Lasso)

2. **Categorical Encoding**: If you have categorical features:
   - One-hot encoding for low cardinality
   - Target encoding for high cardinality
   - Embedding for very high cardinality

3. **Scaling**: Consider:
   - StandardScaler or MinMaxScaler for numeric features
   - Especially important for distance-based algorithms

4. **Model Suggestions**:
   - Random Forest (handles non-linear relationships well)
   - Gradient Boosting (XGBoost, LightGBM, CatBoost)
   - Neural Networks (if you have enough data)
   - Consider class weights if classes are imbalanced
